In [78]:
import pandas as pd
import numpy as np
# pd.options.mode.use_inf_as_na = True
from sklearn import preprocessing


In [79]:
def norm_column(column_data):
    x = column_data.values.astype(float) #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return x_scaled

In [80]:
train_data = pd.read_csv('train.csv')
train_data_copy = train_data.copy()
test_data = pd.read_csv('test.csv')

In [81]:
train_data.drop(columns = ['Id','Street', 'Utilities', 'PoolQC'], inplace = True)
test_with_id = test_data.copy()
test_data.drop(columns = ['Id','Street', 'Utilities', 'PoolQC'], inplace = True)

In [82]:
# Preprocess the data:
# - Convert categorical to one-hot representation
# - NaN
for column in test_data.columns:
    if test_data[column].dtype == object:
        # Need to do one-hot encoding
        new_col = pd.get_dummies(test_data[column].astype(str), dummy_na = False, prefix=column, prefix_sep='_')
        test_data.drop(columns = column, inplace = True)
        test_data = pd.concat([test_data, new_col], axis =1)
    else:
        #normalize
        test_data[column] = norm_column(test_data[[column]])
        # check NaN value
#         if any(train_data[column].isnull()): # replace NaN with median
        median = test_data[~test_data[column].isnull()][column].median()
#         train_data[column] = train_data[column].fillna(median)
        test_data[column].fillna(median, inplace = True)
        #Check inf values
#         if any(np.isinf(train_data[column])): # replace inf with max and -inf with min
#             max_ = train_data[column].max()
#             min_ = train_data[column].min()
#             train_data[column].replace([np.inf, -np.inf], [max_, min_], inplace = True)

In [83]:
for column in train_data.columns:
    if train_data[column].dtype == object:
        # Need to do one-hot encoding
        new_col = pd.get_dummies(train_data[column].astype(str), dummy_na = False, prefix=column, prefix_sep='_')
        train_data.drop(columns = column, inplace = True)
        train_data = pd.concat([train_data, new_col], axis =1)
    else:
        if column != 'SalePrice':
            #normalize
            train_data[column] = norm_column(train_data[[column]])
            # check NaN value
    #         if any(train_data[column].isnull()): # replace NaN with median
        median = train_data[~train_data[column].isnull()][column].median()
    #         train_data[column] = train_data[column].fillna(median)
        train_data[column].fillna(median, inplace = True)
        #Check inf values
#         if any(np.isinf(train_data[column])): # replace inf with max and -inf with min
#             max_ = train_data[column].max()
#             min_ = train_data[column].min()
#             train_data[column].replace([np.inf, -np.inf], [max_, min_], inplace = True)

In [84]:
# for column in train_data.columns:
#     if(any(train_data[column].isnull())):
#         print(train_data[column])
# # len(train_data.columns)

In [85]:
# for column in train_data.columns:
#     if (any(train_data[column].isnull())):
#         print(column)
#     if (any(np.isinf(train_data[column]))):
#         print('Inf!!!!' + column)
#     if (any(np.isinf(-train_data[column]))):
#         print('-Inf!!!!' + column)


In [86]:
# Treat NaN and inf
# Check percentages
len(one_hot_data[one_hot_data['LotFrontage'].isnull()])/len(one_hot_data)
# too many we will not drop
a = one_hot_data.corr(method ='pearson')['LotFrontage']
a[(a>0.5) | (a< -0.5)]
# No correlation with other columns

NameError: name 'one_hot_data' is not defined

In [ ]:
x_train, y_train = train_data.drop(columns= ['SalePrice']), train_data[['SalePrice']]

In [87]:
diff = list(set(x_train.columns) - set(test_data.columns))
for item in diff:
    test_data[item] = 0
    
diff = list(set(test_data.columns) - set(x_train.columns))
test_data = test_data.drop(columns= diff)

In [88]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.01) # higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles
rr.fit(x_train, y_train)
# Ridge_train_score = rr.score(x_train,y_train)
print(x_train.shape)
print(test_data.shape)
prediction = rr.predict(test_data).T[0]

(1460, 296)
(1459, 296)


In [92]:
ids = test_with_id['Id'].values

In [94]:
df = pd.DataFrame({'Id': ids, 'SalePrice': prediction})
df.to_csv('submission.csv', index=False)